In [5]:
import configparser
import csv
import os
import os.path as ops
import pickle

from PIL import Image
import numpy as np
import scipy
import torch

import matplotlib.pyplot as plt
import argparse
import glob
import json
import cv2

import sys
sys.path.append("..")
from dataset.cosmos_dataset import preprocessvideo

In [6]:
outimg_dir = '../../COSMOS_data_tracktor/Track_data_cosmos/v_and_p/traffic_video_GP010615_190724_0545_0715_90sec_calibrated/img1'
ann_txt_path_v = '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP010615_190724_0545_0715_90sec_calibrated_v.txt'
ann_txt_path_p = '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP010615_190724_0545_0715_90sec_calibrated_p.txt'

In [7]:
img_names = []
for img_name in os.listdir(outimg_dir):
    img_names.append(img_name)
img_names.sort()

anns_v = {'{}'.format(i):[] for i in range(1, len(img_names)+1)}
anns_p = {'{}'.format(i):[] for i in range(1, len(img_names)+1)}

with open(ann_txt_path_v,'r') as file:
    data = file.readlines()
    for l in data:
        line = l.split(',')
        line[2] = str(int(float(line[2])))
        line[3] = str(int(float(line[3])))
        line[4] = str(int(float(line[4])))
        line[5] = str(int(float(line[5])))
        anns_v[line[0]].append(line[1:6])

with open(ann_txt_path_p,'r') as file:
    data = file.readlines()
    for l in data:
        line = l.split(',')
        line[2] = str(int(float(line[2])))
        line[3] = str(int(float(line[3])))
        line[4] = str(int(float(line[4])))
        line[5] = str(int(float(line[5])))
        anns_p[line[0]].append(line[1:6])

In [8]:
from tqdm import tqdm
output_dir  = '../../COSMOS_data_tracktor/Tracktor_video'
os.makedirs(output_dir, exist_ok=True)
fps = 10
fourcc = cv2.VideoWriter_fourcc(*'XVID')
videoWriter = cv2.VideoWriter(ops.join(output_dir, ops.split(ops.split(outimg_dir)[0])[1]+'.mp4'), fourcc, fps, (1920,1080))
for i in tqdm(range(len(img_names))):
    img = cv2.imread(ops.join(outimg_dir, img_names[i]))
    for box in anns_v[str(i+1)]:
        img = cv2.rectangle(img, (int(box[1]),int(box[2])), (int(box[1])+int(box[3]),int(box[2])+int(box[4])), (0,255,0), 2)
        img = cv2.putText(img, 'v:'+box[0], (int(box[1]),int(box[2])), cv2.FONT_HERSHEY_SIMPLEX , 0.6, (0, 255, 0) , 2, cv2.LINE_AA) 
    for box in anns_p[str(i+1)]:
        img = cv2.rectangle(img, (int(box[1]),int(box[2])), (int(box[1])+int(box[3]),int(box[2])+int(box[4])), (255,0,0), 2)
        img = cv2.putText(img, 'p:'+box[0], (int(box[1]),int(box[2])), cv2.FONT_HERSHEY_SIMPLEX , 0.6, (255, 0, 0) , 2, cv2.LINE_AA)     
    videoWriter.write(img)
videoWriter.release()

100%|██████████| 899/899 [00:44<00:00, 20.13it/s]
